<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_04_5_memory_persist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 4: LangChain: Chat and Memory**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 4 Material

* Part 4.1: LangChain Conversations [[Video]](https://www.youtube.com/watch?v=Effbhxq07Ag) [[Notebook]](t81_559_class_04_1_langchain_chat.ipynb)
* Part 4.2: Conversation Buffer Window Memory [[Video]](https://www.youtube.com/watch?v=14RgiFVGfAA) [[Notebook]](t81_559_class_04_2_memory_buffer.ipynb)
* Part 4.3: Conversation Token Buffer Memory [[Video]](https://www.youtube.com/watch?v=QTe5g2c3bSM) [[Notebook]](t81_559_class_04_3_memory_token.ipynb)
* Part 4.4: Conversation Summary Memory [[Video]](https://www.youtube.com/watch?v=asZQ8Ktqmt8) [[Notebook]](t81_559_class_04_4_memory_summary.ipynb)
* **Part 4.5: Persisting Langchain Memory** [[Video]](https://www.youtube.com/watch?v=sjCyqqOQcPA) [[Notebook]](t81_559_class_04_5_memory_persist.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: not using Google CoLab


# 4.5: Persisting Langchain Memory

In this section, we delve into the creation of an innovative utility class designed for managing chat interactions—aptly named ChatBot. This class stands as a cornerstone for applications that require robust handling of chat history, including features to load and save these interactions, as well as capabilities to undo or regenerate outputs. The architecture of ChatBot integrates advanced language models with structured templates to facilitate interactive conversations, ensuring a seamless and dynamic user experience.

The core functionality of ChatBot revolves around its sophisticated integration of components from the langchain ecosystem. It utilizes a large language model for generating chat responses and another for summarizing conversations, complemented by a template that structures each interaction. Key features include maintaining an editable conversation history, implementing undo and regenerate operations for greater control over chat flows, and mechanisms to persist conversation states, enhancing the utility and flexibility of the chat interface. As we explore ChatBot, we will uncover how each component contributes to its overall capability to manage complex conversational contexts effectively.

The code for this class follows.

In [3]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown
import pickle

DEFAULT_TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""

MODEL = 'gpt-4o-mini'

class ChatBot:
    def __init__(self, llm_chat, llm_summary, template):
        """
        Initializes the ChatBot with language models and a template for conversation.

        :param llm_chat: A large language model for handling chat responses.
        :param llm_summary: A large language model for summarizing conversations.
        :param template: A string template defining the conversation structure.
        """
        self.llm_chat = llm_chat
        self.llm_summary = llm_summary
        self.template = template
        self.prompt_template = PromptTemplate(input_variables=["history", "input"], template=self.template)

        # Initialize memory and conversation chain
        self.memory = ConversationSummaryMemory(llm=self.llm_summary)
        self.conversation = ConversationChain(
            prompt=self.prompt_template,
            llm=self.llm_chat,
            memory=self.memory,
            verbose=False
        )

        self.history = []

    def converse(self, prompt):
        """
        Processes a conversation prompt and updates the internal history and memory.

        :param prompt: The input prompt from the user.
        :return: The generated response from the language model.
        """
        self.history.append([self.memory.buffer, prompt])
        output = self.conversation.invoke(prompt)
        return output['response']

    def chat(self, prompt):
        """
        Handles the full cycle of receiving a prompt, processing it, and displaying the result.

        :param prompt: The input prompt from the user.
        """
        print(f"Human: {prompt}")
        output = self.converse(prompt)
        display_markdown(output, raw=True)

    def print_memory(self):
        """
        Displays the current state of the conversation memory.
        """
        print("**Memory:")
        print(self.memory.buffer)

    def clear_memory(self):
        """
        Clears the conversation memory.
        """
        self.memory.clear()

    def undo(self):
        """
        Reverts the conversation memory to the state before the last interaction.
        """
        if len(self.history) > 0:
            self.memory.buffer = self.history.pop()[0]
        else:
            print("Nothing to undo.")

    def regenerate(self):
        """
        Re-executes the last undone interaction, effectively redoing an undo operation.
        """
        if len(self.history) > 0:
            self.memory.buffer, prompt = self.history.pop()
            self.chat(prompt)
        else:
            print("Nothing to regenerate.")

    def save_history(self, file_path):
        """
        Saves the conversation history to a file using pickle.

        :param file_path: The file path where the history should be saved.
        """
        with open(file_path, 'wb') as f:
            pickle.dump(self.history, f)

    def load_history(self, file_path):
        """
        Loads the conversation history from a file using pickle.

        :param file_path: The file path from which to load the history.
        """
        with open(file_path, 'rb') as f:
            self.history = pickle.load(f)
            # Optionally reset the memory based on the last saved state
            if self.history:
                self.memory.buffer = self.history[-1][0]



## Testing the Chat Bot
Before we see how this class was constructed, lets see how you can use it. The following shows a simple chat interaction.

In [4]:
MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature= 0.3,
  n= 1)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)

c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.chat("What is my name.")
c.chat("Do I like coffee?")




C:\Users\yrobi\AppData\Local\Temp\ipykernel_11532\2203850189.py:33: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  self.conversation = ConversationChain(


Human: Hello, my name is Jeff.


Hello, Jeff! How can I assist you today?

Human: I like coffee.


That's great to hear, Jeff! Coffee can be such a delightful beverage. Do you have a favorite type of coffee or a preferred way to prepare it?

Human: What is my name.


Your name is Jeff. How can I assist you today?

Human: Do I like coffee?


Yes, you mentioned earlier that you like coffee! Do you have a favorite type or a preferred way to prepare it?

## Load and Save Chat Memory

The chat bot allows the complete chat history and memory to be saved and loaded from a file.

In [5]:
MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature= 0.3,
  n= 1)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)
c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.chat("What is my name.")
c.chat("Do I like coffee?")

Human: Hello, my name is Jeff.


Hello, Jeff! How can I assist you today?

Human: I like coffee.


That's great to hear, Jeff! Coffee can be a wonderful way to start the day or take a break. Do you have a favorite type of coffee or a preferred brewing method?

Human: What is my name.


Your name is Jeff. How can I assist you today, Jeff?

Human: Do I like coffee?


Yes, you mentioned that you like coffee! What's your favorite type or brewing method?

We now save the chat memory.

In [6]:
c.save_history('history.pkl')

You can now reload the chat memory; the bot should remember the discussion.

In [7]:
c = ChatBot(llm, llm, DEFAULT_TEMPLATE)
c.load_history('history.pkl')
c.chat("What is my name?")

Human: What is my name?


Your name is Jeff. How can I assist you today, Jeff?

## Undoing and Regenerating the Chat

The chatbot allows us to undo parts of the conversation, and it forgets. We can also regenerate outputs to get a better response.

In [8]:
MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature= 0.3,
  n= 1)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)

c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.undo()
c.chat("What is my name.")
c.chat("Do I like coffee?")
c.regenerate()



Human: Hello, my name is Jeff.


Hello, Jeff! How can I assist you today?

Human: I like coffee.


That's great to hear, Jeff! Coffee can be such a delightful way to start the day or take a break. Do you have a favorite type of coffee or a preferred brewing method?

Human: What is my name.


Your name is Jeff. How can I assist you today?

Human: Do I like coffee?


That's a great question, Jeff! I don't have access to your personal preferences, but if you enjoy the rich flavor and aroma of coffee, then you might like it. Do you usually drink coffee, or are you considering trying it?

Human: Do I like coffee?


That's a great question, Jeff! Do you enjoy coffee, or are you more of a tea person? If you have a preference, I'd love to hear about it!

## Implementing the Chatbot

In this section, we delve into the architecture of a sophisticated chatbot built using the LangChain and OpenAI libraries, encapsulated within the ChatBot class. The ChatBot class leverages two language models and a structured conversation template to engage users in a dynamic conversational flow.

The class initiates by defining its basic components: llm_chat and llm_summary are large language models used for chat responses and conversation summarization, respectively. The template parameter defines the conversation's structure, guiding how interactions are formatted and presented. This template incorporates markdown formatting and placeholders for the conversation history and the latest user input, facilitating a clear and structured dialogue presentation.

Upon instantiation, the ChatBot initializes various components. It constructs a PromptTemplate, which prepares the structure for input variables and the conversation template. The ConversationSummaryMemory, initialized with the llm_summary, acts as a dynamic storage for conversation excerpts, aiding in recalling context or summarizing past dialogues. This memory component is crucial for maintaining conversation continuity, especially in sessions that might span multiple interactions.

The ConversationChain is another pivotal element, combining the prompt template with the llm_chat model to process and respond to user inputs. The chain's verbose mode is set to False to streamline output without extraneous system messages.

Interaction with the ChatBot is primarily through the chat method, where users provide a prompt, and the bot displays the response after processing. Internally, the converse method updates the conversation history and uses the conversation chain to generate a response. This method's utility lies in its ability to append each interaction to the history log, ensuring all discourse components are retained for future reference or undo actions.

Memory management functions like print_memory, clear_memory, and undo provide users and developers with tools to view, reset, or revert the conversation state, thus offering flexibility in managing the dialogue flow and content. Additional functionalities to save and load conversation history are provided through save_history and load_history, utilizing Python’s pickle module for object serialization.

This class structure not only facilitates robust conversation handling and memory management but also showcases the integration of advanced language models in practical applications, demonstrating the potential of AI in enhancing interactive software solutions. Through such a detailed setup, ChatBot serves as a comprehensive framework for building interactive, memory-aware chatbot systems.